# Select the LCLS-II py3 kernel in the top right
# Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import psana as ps

# Specify experiment and run number. Then generate datasource

In [ ]:
exp = 'tmolv2918'
run_number = 215
ds = ps.DataSource(exp=exp, run=run_number)
run = next(ds.runs())

# Print detectors available in this (exp, run)
Fast detectors. Make a measurement every shot

In [ ]:
run.detnames

# Print epics detectors available
These are slow detectors. While they write a value for every event, they do not update at 120Hz.

In [ ]:
run.epicsinfo

# Initialize detector objects to read out shot by shot measuremnets

In [ ]:
gmd = run.Detector('gmd') # xray pulse energy monitor
timing = run.Detector('timing') # event codes
opal = run.Detector('tmo_atmopal') # timetool
hsd = run.Detector('hsd') # ion tof
ebeam = run.Detector('ebeam') # ebeam parameters
und42K = run.Detector('UND_39_k')

# Test data access for a detector
Use a test evt to see what the detector object returns
Many of the detectors call functions are undocumented. Try typing '<Detector Obj>.' then pressing tab to determine the call function. For example,
```python
gmd.[tab]
gmd.raw.[tab]
gmd.raw.energy
help(gmd.raw.energy)
```
shows you that the xray pulse energy may be read by `gmd.raw.energy(evt0)`

In [ ]:
evt0=next( run.events() )

In [ ]:
help(gmd.raw.energy)

In [ ]:
gmd.raw.energy(evt0)

# Loop over events to read in data
## This script provides an example of how to read in the detectors from the xtc files using the detector and run objects you've generated above. For a more streamlined approach, see read_raw_streamlined.ipynb

nupdate := Specifies when to print out updates

nskip := Loop skips events modulo nskip

Nfind := Loop stops once this number of events is read out

In [ ]:
# User specified inputs
Nfind = 300
nskip = 2
nupdate = 50
channel = 0 # ion ToF

# Initialize arrays
energies = np.empty(Nfind)
photonEs = np.empty(Nfind)
undKs = np.empty(Nfind)
evrs = None
t = None
ims = None
wfs = None

# Event iteration
Nfound = 0
for nevent, event in enumerate(run.events()):
    
    if nevent%nupdate==0: print(nevent, Nfound)
    if nevent%nskip!=0: continue
    
    undK = und42K(event)
    if undK is None: continue
    energy = gmd.raw.energy(event)
    if energy is None:
        print("Didn't find GMD")
        continue
    evr = timing.raw.eventcodes(event)
    if evr is None:
        print("Didn't find EVR")
        continue
    im = opal.raw.image(event)
    if im is None:
        print("Didn't find Opal")
        continue
    hsd_data = hsd.raw.waveforms(event)
    if hsd_data is None:
        print("Didn't find HSD")
        continue
    photonE = ebeam.raw.ebeamPhotonEnergy(event)
    if photonE is None:
        print("Didn't find photon energy")
        continue
    
    energies[Nfound] = energy
    photonEs[Nfound] = photonE
    undKs[Nfound] = undK
    
    if evrs is None:
        evrs = np.empty((Nfind, len(evr)))
    evrs[Nfound] = evr
    
    if t is None:
        t = hsd_data[channel]['times'][::4] * 1e6
        wfs = np.empty((Nfind, len(t)))
    wfs[Nfound] = hsd_data[channel][0][::4]
    
    if ims is None:
        ims = np.empty((Nfind,)+im.shape)
    ims[Nfound] = im
    
    Nfound += 1
    if Nfound==Nfind: break

# Example plots and analyses
## Histogram of pulse energies

In [ ]:
plt.hist(energies * 1e3, bins=20);
plt.xlabel("Pulse energy / uJ");

## Averaged OPAL image

In [ ]:
plt.imshow(ims.mean(0))
plt.colorbar()
plt.title("Averaged OPAL image");

## TOF traces

In [ ]:
gas_on = evrs[:,70]==1
gas_off = np.logical_not(gas_on)

plt.plot(t, wfs[gas_on].mean(0), 'k', label='Jet on');
plt.plot(t, wfs[gas_off].mean(0), 'r', alpha=0.5, label='Jet off');
plt.title("5 ion TOF traces");
plt.legend()
plt.xlabel("ToF / us")